### Load tensorflow

In [2]:
import tensorflow as tf

C:\Users\SUPERMAN1\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SUPERMAN1\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SUPERMAN1\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\SUPERMAN1\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Futu

In [170]:
import numpy as np 

### Enable Eager Execution if you are using tensorflow 1.x

In [171]:
tf.enable_eager_execution()

### Collect Data

In [172]:
import pandas as pd

In [173]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [174]:
cols = data.columns
print(cols)

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


### Drop columns `date` and  `symbol`

In [175]:
data = data.drop(["date", "symbol"], axis=1)

In [176]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [177]:
data = data.head(1000)

In [178]:
data.shape

(1000, 5)

In [179]:
data.dtypes

open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Convert Float64 to Float32

In [180]:
data['open'] = np.float32(data['open'])
data['close'] = np.float32(data['close'])
data['low'] = np.float32(data['low'])
data['high'] = np.float32(data['high'])
data['volume'] = np.float32(data['volume'])

In [181]:
data.dtypes

open      float32
close     float32
low       float32
high      float32
volume    float32
dtype: object

In [182]:
x = data.drop(columns = "close",axis=1)
y=data['close']

### Divide the data into train and test sets

In [183]:
# from sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,test_size=0.3,random_state=123)

### Normalize Train and Test Data 

In [184]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
x= transformer.fit_transform(x)
y= np.array(y)


train_x, test_x,train_y,test_y = train_test_split(x, y, test_size=.3, random_state=7)

## Building the graph in tensorflow

2.Define Weights and Bias

In [185]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [186]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [187]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [188]:
def train(x, y_train, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_train, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [189]:
for i in range(100):
    
    w, b = train(train_x, train_y, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, 
          loss(train_y, prediction(train_x, w, b)).numpy())

Current Loss on iteration 0 8002.651
Current Loss on iteration 1 7663.5674
Current Loss on iteration 2 7351.068
Current Loss on iteration 3 7063.063
Current Loss on iteration 4 6797.648
Current Loss on iteration 5 6553.0317
Current Loss on iteration 6 6327.598
Current Loss on iteration 7 6119.837
Current Loss on iteration 8 5928.3623
Current Loss on iteration 9 5751.9053
Current Loss on iteration 10 5589.2764
Current Loss on iteration 11 5439.4004
Current Loss on iteration 12 5301.2764
Current Loss on iteration 13 5173.976
Current Loss on iteration 14 5056.6597
Current Loss on iteration 15 4948.5464
Current Loss on iteration 16 4848.9014
Current Loss on iteration 17 4757.0674
Current Loss on iteration 18 4672.435
Current Loss on iteration 19 4594.4414
Current Loss on iteration 20 4522.5615
Current Loss on iteration 21 4456.3125
Current Loss on iteration 22 4395.264
Current Loss on iteration 23 4338.998
Current Loss on iteration 24 4287.1445
Current Loss on iteration 25 4239.3525
Curren

### Get the shapes and values of W and b

In [190]:
w.shape

TensorShape([Dimension(4), Dimension(1)])

In [191]:
b.shape

TensorShape([Dimension(1)])

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [192]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1,  input_shape=(4,)))

### Execute the model

In [193]:
#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

In [ ]:
train_x, test_x,train_y,test_y = train_test_split(x1, y1, test_size=.3, random_state=7)

### Classification using Keras 

In [195]:
model.fit(train_x,train_y, epochs=100)

Epoch 1/100
700/700 [==============================] - 0s 307us/sample - loss: 5247.4612
Epoch 2/100
700/700 [==============================] - 0s 43us/sample - loss: 5420.1366
Epoch 3/100
700/700 [==============================] - 0s 49us/sample - loss: 4216.2470
Epoch 4/100
700/700 [==============================] - 0s 44us/sample - loss: 4605.9146
Epoch 5/100
700/700 [==============================] - 0s 57us/sample - loss: 4511.6285
Epoch 6/100
700/700 [==============================] - 0s 50us/sample - loss: 3745.7957
Epoch 7/100
700/700 [==============================] - 0s 52us/sample - loss: 3984.0476
Epoch 8/100
700/700 [==============================] - 0s 46us/sample - loss: 4058.1979
Epoch 9/100
700/700 [==============================] - 0s 46us/sample - loss: 3977.3939
Epoch 10/100
700/700 [==============================] - 0s 44us/sample - loss: 3774.0420
Epoch 11/100
700/700 [==============================] - 0s 43us/sample - loss: 4211.9756
Epoch 12/100
700/700 [=======

700/700 [==============================] - 0s 40us/sample - loss: 3678.9661
Epoch 94/100
700/700 [==============================] - 0s 47us/sample - loss: 3678.6678
Epoch 95/100
700/700 [==============================] - 0s 44us/sample - loss: 3681.6759
Epoch 96/100
700/700 [==============================] - 0s 39us/sample - loss: 3678.6400
Epoch 97/100
700/700 [==============================] - 0s 44us/sample - loss: 3682.6780
Epoch 98/100
700/700 [==============================] - 0s 44us/sample - loss: 3678.7523
Epoch 99/100
700/700 [==============================] - 0s 47us/sample - loss: 3678.9947
Epoch 100/100
700/700 [==============================] - 0s 44us/sample - loss: 3683.1414


In [196]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_v1_4 (Ba multiple                  16        
_________________________________________________________________
dense_11 (Dense)             multiple                  5         
Total params: 21
Trainable params: 13
Non-trainable params: 8
_________________________________________________________________


### Load the given Iris data using pandas (Iris.csv)

In [197]:
iris_df = pd.read_csv('Iris.csv')

### Splitting the data into feature set and target set

In [198]:
features = iris_df.drop(columns= ['Species']) 


target = iris_df[['Species']]

In [199]:
features.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
dtype: object

In [200]:

features.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,1,5.1,3.5,1.4,0.2
1,2,4.9,3.0,1.4,0.2
2,3,4.7,3.2,1.3,0.2
3,4,4.6,3.1,1.5,0.2
4,5,5.0,3.6,1.4,0.2


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [201]:
from sklearn.preprocessing import LabelEncoder
Y_processed = pd.DataFrame(target).apply(LabelEncoder().fit_transform)
testY = tf.keras.utils.to_categorical(Y_processed, num_classes=3)

### Divide the dataset into Training and test (70:30)

In [202]:
from sklearn.model_selection import train_test_split

train_x, test_x,train_y,test_y = train_test_split(features, testY, test_size=.3, random_state=7)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [203]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [204]:
model.fit(train_x.values, train_y, validation_split=0.2, epochs=30)

Train on 84 samples, validate on 21 samples
Epoch 1/30
84/84 [==============================] - 0s 5ms/sample - loss: 1.0580 - acc: 0.2381 - val_loss: 1.4069 - val_acc: 0.2857
Epoch 2/30
84/84 [==============================] - 0s 96us/sample - loss: 1.0044 - acc: 0.3095 - val_loss: 1.4471 - val_acc: 0.2857
Epoch 3/30
84/84 [==============================] - 0s 131us/sample - loss: 0.9689 - acc: 0.4167 - val_loss: 1.5120 - val_acc: 0.2857
Epoch 4/30
84/84 [==============================] - 0s 95us/sample - loss: 0.9341 - acc: 0.4643 - val_loss: 1.5495 - val_acc: 0.2857
Epoch 5/30
84/84 [==============================] - 0s 131us/sample - loss: 0.8933 - acc: 0.5119 - val_loss: 1.5785 - val_acc: 0.2857
Epoch 6/30
84/84 [==============================] - 0s 167us/sample - loss: 0.8806 - acc: 0.5357 - val_loss: 1.6010 - val_acc: 0.2857
Epoch 7/30
84/84 [==============================] - 0s 131us/sample - loss: 0.8561 - acc: 0.5595 - val_loss: 1.6371 - val_acc: 0.2857
Epoch 8/30
84/84 [====

In [205]:
model.evaluate(test_x,test_y)

45/45 [==============================] - 0s 112us/sample - loss: 1.1408 - acc: 0.6444


[1.140827516714732, 0.64444447]

### Report Accuracy of the predicted values

In [169]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_v1_3 (Ba multiple                  20        
_________________________________________________________________
dense_8 (Dense)              multiple                  60        
_________________________________________________________________
dense_9 (Dense)              multiple                  88        
_________________________________________________________________
dense_10 (Dense)             multiple                  27        
Total params: 195
Trainable params: 185
Non-trainable params: 10
_________________________________________________________________
